In [1]:
from jpype import JClass, getDefaultJVMPath, shutdownJVM, startJVM, JString
import os
import string
import pandas as pd #to read csv files
import random 
import numpy as np #handle operations on arrays easily

In [2]:
# define paths
ZEMBEREK_PATH = r"C:/Users/hp/Desktop/text_summarization/jars/zemberek-full/"
DATA_PATH = "data"


In [7]:
        startJVM(
            getDefaultJVMPath(),
            '-ea',
            f'-Djava.class.path={ZEMBEREK_PATH}',
            convertStrings=False
        )

In [8]:
TurkishTokenizer: JClass = JClass("zemberek.tokenization.TurkishTokenizer")
TurkishMorphology: JClass = JClass("zemberek.morphology.TurkishMorphology")
TurkishSentenceNormalizer: JClass = JClass(
    "zemberek.normalization.TurkishSentenceNormalizer"
)
Paths: JClass = JClass("java.nio.file.Paths")
morphology = TurkishMorphology.createWithDefaults()
tokenizer = TurkishTokenizer.DEFAULT
normalizer = TurkishSentenceNormalizer(
    TurkishMorphology.createWithDefaults(),
    Paths.get(str(os.path.join(DATA_PATH, "normalization"))),
    Paths.get(str(os.path.join(DATA_PATH, "lm", "lm.2gram.slm"))),
)

In [11]:
#from the csv file read all the lines and make a table
dataset = pd.read_csv("C:/Users/hp/Desktop/text_summarization/data/7all.csv", encoding='utf-8', delimiter=',', names=["cat", "text"])

print("dataset has",len(dataset), "lines")


dataset has 4900 lines


In [12]:
samples = dataset.sample(3)
print(samples)

             cat                                               text
4359  teknoloji    işte akıllı telefon piyasasının 2012 verileri...
1688    ekonomi    ihalede matematiği yanlış yapmışız istanbul_p...
1967    ekonomi    gişelerin hepsini kaldırabiliriz ptt genel_mü...


In [13]:
print(samples.iloc[0].text)

 işte akıllı telefon piyasasının 2012 verileri ! telefon piyasasında büyük bir hareketlilik var ve işte bu hareketliliğin sayılarla 2011 ve 2012 verileri teknolojik araştırmalar ve danışmanlık kuruluşu gartner kısa süre önce son yayınladığı çalışmasında 2012 yılı mobil telefon satış istatistiklerini duyurdu işte veriler bu tabloda şirketlerin 2011 ve 2012 yıllarında sattıkları mobil telefon adetleri ve pazar payları görülüyor lider samsung ! nokia 2011 yılında yüzde 23 8 lik pazar payının yüzde 20 sini kaybedip yüzde 19 1 e gerilerken samsung yüzde 20 lik büyümeyle pazar payındaki liderliği ele geçirdi apple tırmanıyor ! listede en dikkat çekici nokta apple ın 2011 yılına göre pazar payındaki yükselişi verilere göre apple 2011 yılına göre yüzde 33 3 oranında pazar payını artırmış görünüyor bu tabloda ise şirketlerin 2011 ve 2012 yıllarının 4 çeyreklerindeki pazar payları ve satılan telefon adedi görülmekte iphone 5 çok tutuldu ! 2012 nin 4 çeyreğinde piyasaya sürülen iphone 5 in pazar 

In [14]:
cats = dataset.groupby("cat").size()
print(cats)

cat
dunya         700
ekonomi       700
kultur        700
saglik        700
siyaset       700
spor          700
teknoloji     700
dtype: int64


In [16]:
normalized_text = str(normalizer.normalize(JString(text)))
normalized_text

'google bu kez edward_görey dedi ! google bu kez de ünlü illüstratör yazar ve sair edward_görey için çok özel doodle hazırladı googleın in 88 doğum gününe özel doodle yaptığı edward_görey 1925 yılında doğmuş nisan 2000 de hayata veda etmiştir edward_görey bir gothiç art çizeridir tim_burton un bu sanatçının çizimlerinden etkilendiği şeklinde bir söylenti vardır göre bram_stoker hg wells ts eliot gibi büyük yazarların kitaplarına illüstrasyonlar yaptı ayrıca sonraki yıllarda bir çok popüler çocuk kitabına çizimler yapmıştır gören bir illüstratör olarak tarif edilir onun kitapları büyük kitapçılarda ve mizah ve karikatür bölümlerde bulunur göre 100 den fazla kitap yazdı göre 75 yaşında öldü'

In [18]:
#Noktalama işaretleri kaldırıldı.
punctuation_free = "".join([i for i in normalized_text if i not in string.punctuation])
punctuation_free

'google bu kez edwardgörey dedi  google bu kez de ünlü illüstratör yazar ve sair edwardgörey için çok özel doodle hazırladı googleın in 88 doğum gününe özel doodle yaptığı edwardgörey 1925 yılında doğmuş nisan 2000 de hayata veda etmiştir edwardgörey bir gothiç art çizeridir timburton un bu sanatçının çizimlerinden etkilendiği şeklinde bir söylenti vardır göre bramstoker hg wells ts eliot gibi büyük yazarların kitaplarına illüstrasyonlar yaptı ayrıca sonraki yıllarda bir çok popüler çocuk kitabına çizimler yapmıştır gören bir illüstratör olarak tarif edilir onun kitapları büyük kitapçılarda ve mizah ve karikatür bölümlerde bulunur göre 100 den fazla kitap yazdı göre 75 yaşında öldü'

In [20]:
#Sayılar kaldırıldı.
digit_free = ''.join([i for i in punctuation_free if not i.isdigit()])
digit_free

'google bu kez edwardgörey dedi  google bu kez de ünlü illüstratör yazar ve sair edwardgörey için çok özel doodle hazırladı googleın in  doğum gününe özel doodle yaptığı edwardgörey  yılında doğmuş nisan  de hayata veda etmiştir edwardgörey bir gothiç art çizeridir timburton un bu sanatçının çizimlerinden etkilendiği şeklinde bir söylenti vardır göre bramstoker hg wells ts eliot gibi büyük yazarların kitaplarına illüstrasyonlar yaptı ayrıca sonraki yıllarda bir çok popüler çocuk kitabına çizimler yapmıştır gören bir illüstratör olarak tarif edilir onun kitapları büyük kitapçılarda ve mizah ve karikatür bölümlerde bulunur göre  den fazla kitap yazdı göre  yaşında öldü'

In [21]:
tokens = []
for i, token in enumerate(tokenizer.tokenizeToStrings(JString(digit_free))):
    tokens.append(str(token))
tokens

['google',
 'bu',
 'kez',
 'edwardgörey',
 'dedi',
 'google',
 'bu',
 'kez',
 'de',
 'ünlü',
 'illüstratör',
 'yazar',
 've',
 'sair',
 'edwardgörey',
 'için',
 'çok',
 'özel',
 'doodle',
 'hazırladı',
 'googleın',
 'in',
 'doğum',
 'gününe',
 'özel',
 'doodle',
 'yaptığı',
 'edwardgörey',
 'yılında',
 'doğmuş',
 'nisan',
 'de',
 'hayata',
 'veda',
 'etmiştir',
 'edwardgörey',
 'bir',
 'gothiç',
 'art',
 'çizeridir',
 'timburton',
 'un',
 'bu',
 'sanatçının',
 'çizimlerinden',
 'etkilendiği',
 'şeklinde',
 'bir',
 'söylenti',
 'vardır',
 'göre',
 'bramstoker',
 'hg',
 'wells',
 'ts',
 'eliot',
 'gibi',
 'büyük',
 'yazarların',
 'kitaplarına',
 'illüstrasyonlar',
 'yaptı',
 'ayrıca',
 'sonraki',
 'yıllarda',
 'bir',
 'çok',
 'popüler',
 'çocuk',
 'kitabına',
 'çizimler',
 'yapmıştır',
 'gören',
 'bir',
 'illüstratör',
 'olarak',
 'tarif',
 'edilir',
 'onun',
 'kitapları',
 'büyük',
 'kitapçılarda',
 've',
 'mizah',
 've',
 'karikatür',
 'bölümlerde',
 'bulunur',
 'göre',
 'den',
 'fazla

In [22]:
from collections import Counter

all_words = []
for idx,rows in dataset.iterrows():
    text = rows.text
    all_words.extend(text.split(" "))

words_freq = Counter(all_words)

print(words_freq)

Counter({'ve': 27180, 'bir': 20463, 'bu': 13527, 'da': 12317, '\x92': 11908, 'de': 11596, 'için': 7813, 'ile': 7088, 'türkiye': 5475, 'çok': 5108, 'nin': 5071, 'olarak': 5058, '': 4900, '\x93': 4577, '\x94': 4573, 'daha': 4326, 'en': 4291, 'ın': 4008, 'olan': 3872, 'ise': 3493, 'olduğunu': 3194, 'in': 3174, 'dedi': 2977, 'gibi': 2888, '1': 2869, 'nın': 2869, '2': 2832, 'yüzde': 2790, '3': 2674, 'kadar': 2647, 'ilk': 2623, 'yeni': 2600, 'büyük': 2577, 'her': 2523, 'a': 2337, 'sonra': 2282, '5': 2233, 'bin': 2208, 'söyledi': 2202, 'ya': 2152, 'son': 2141, 'önemli': 2134, 'var': 2126, 'ilgili': 2113, 'göre': 2112, 'yıl': 2031, 'milyon': 2019, 'etti': 1887, 'ne': 1856, '4': 1845, 'yer': 1839, 'tarafından': 1834, 'ifade': 1816, 'iyi': 1808, 'olduğu': 1791, 'diye': 1781, 'devam': 1764, 'e': 1764, 'eden': 1724, '10': 1715, 'ancak': 1702, 'o': 1674, 'yapılan': 1640, 'iki': 1640, 'ye': 1634, 'ama': 1592, 'önce': 1588, 'konuştu': 1565, 'arasında': 1543, 'istanbul': 1443, '/': 1435, 'oldu': 1430,

In [23]:
with open(str(os.path.join(DATA_PATH, "stop-words.txt"))) as file:
    stopwords_zemberek = [line.rstrip() for line in file]
len(stopwords_zemberek)

1797

In [24]:
#string of all turkish characters 
turkishCharaters = "abcçdefgğhıijklmnoöprsştuüvyz_"
#function use the 'turkishCharaters' list to eliminate symbols and strange characters.
def RealWord(string):
    if not any(c not in turkishCharaters for c in string):
        return True
    else:
        return False

In [25]:
#we will remove all words that has low frequency 
threshold = 30
words_freq = dict(filter(lambda x: x[1]>threshold, words_freq.items()))
#take all words from the dict
words = list(words_freq.keys())
#remove stop words.
filtered_words =  [i for i in tokens if i not in stopwords_zemberek]
#remove non turkish characters.
filtered_words = list(filter(RealWord, filtered_words))[1:]
print("the number of words is",len(filtered_words),"word, after we cleaned it.")
print("-"*50)
print(filtered_words)

the number of words is 76 word, after we cleaned it.
--------------------------------------------------
['kez', 'dedi', 'google', 'kez', 'ünlü', 'illüstratör', 'yazar', 'sair', 'için', 'çok', 'özel', 'doodle', 'hazırladı', 'googleın', 'in', 'doğum', 'gününe', 'özel', 'doodle', 'yaptığı', 'yılında', 'doğmuş', 'nisan', 'hayata', 'veda', 'etmiştir', 'gothiç', 'art', 'çizeridir', 'timburton', 'un', 'sanatçının', 'çizimlerinden', 'etkilendiği', 'şeklinde', 'söylenti', 'vardır', 'göre', 'bramstoker', 'hg', 'ts', 'eliot', 'büyük', 'yazarların', 'kitaplarına', 'illüstrasyonlar', 'yaptı', 'ayrıca', 'sonraki', 'yıllarda', 'çok', 'popüler', 'çocuk', 'kitabına', 'çizimler', 'yapmıştır', 'gören', 'illüstratör', 'olarak', 'tarif', 'edilir', 'onun', 'kitapları', 'büyük', 'kitapçılarda', 'mizah', 'karikatür', 'bölümlerde', 'bulunur', 'göre', 'den', 'kitap', 'yazdı', 'göre', 'yaşında', 'öldü']


In [26]:
#we just need to filter it with our new keys(filterd words)
new_freq = dict(filter(lambda x: x[0] in filtered_words, words_freq.items()))
#just sort words dict according to its frequencies 
new_freq = dict(sorted(new_freq.items(), key=lambda x: x[1], reverse=True))
print(new_freq)

{'için': 7813, 'çok': 5108, 'olarak': 5058, 'in': 3174, 'dedi': 2977, 'büyük': 2577, 'göre': 2112, 'un': 1402, 'yaptığı': 1325, 'den': 1215, 'özel': 1168, 'yılında': 1003, 'kez': 967, 'ayrıca': 703, 'yaptı': 573, 'çocuk': 453, 'şeklinde': 449, 'ünlü': 400, 'google': 336, 'onun': 325, 'doğum': 321, 'kitap': 312, 'yıllarda': 262, 'hayata': 259, 'vardır': 249, 'sonraki': 189, 'nisan': 163, 'yaşında': 162, 'gören': 147, 'yazar': 139, 'yazdı': 99, 'öldü': 86, 'popüler': 85, 'doodle': 64, 'sanatçının': 62, 'veda': 58, 'art': 45, 'edilir': 44, 'kitapları': 38, 'hazırladı': 38, 'etmiştir': 33}


In [27]:
print("for the first ten words:\n")
print("filtered_words:",filtered_words[:10])
print("-"*50)
init_vector = [0]*len(filtered_words)
print("init_vector:",init_vector[:10])

for the first ten words:

filtered_words: ['kez', 'dedi', 'google', 'kez', 'ünlü', 'illüstratör', 'yazar', 'sair', 'için', 'çok']
--------------------------------------------------
init_vector: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [28]:
#[0,0,0....0] ==> [1,4,2....2]
def create_vector(text):
    words_of_text = text.split(" ")
    text_vector = [0]*len(init_vector)
    for word in words_of_text:
        if word in filtered_words:
            idx = filtered_words.index(word)
            text_vector[idx] += 1
    return text_vector

all_vectors = []

#loop through all texts
for idx,row in dataset.iterrows():
    text = dataset.loc[idx, "text"]
    all_vectors.append(create_vector(text))


print("we now have",len(all_vectors),"vector.")
print("these vectors will be our input data to our model.")
print("-"*50)
print("first vector:",all_vectors[0])

we now have 4900 vector.
these vectors will be our input data to our model.
--------------------------------------------------
first vector: [0, 5, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [29]:
labels = dataset["cat"].tolist()

model_dataset = list(zip(all_vectors,labels))

print("one data sample:",model_dataset[0])

one data sample: ([0, 5, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'siyaset ')


In [30]:
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
model_dataset_shuf = shuffle(model_dataset)
all_vectors_shuf,labels_shuf = zip(*model_dataset_shuf)

data_length = len(all_vectors)
train_x,train_y = all_vectors_shuf[:int(data_length*2/3)],labels_shuf[:int(data_length*2/3)]
test_x,test_y = all_vectors_shuf[int(data_length*2/3):],labels_shuf[int(data_length*2/3):]


print("total data size:",data_length)
print("Train data size:",len(train_x))
print("Test data size:",len(test_x))

total data size: 4900
Train data size: 3266
Test data size: 1634


In [33]:
print ("Training the random forest...")

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# This may take a few minutes to run
forest = forest.fit( train_x, train_y )

print("training is finished.")

Training the random forest...
training is finished.


In [34]:
from sklearn import metrics

def cal_acc(set_x,set_y):
    results = forest.predict(set_x)
    cnf_matrix = metrics.confusion_matrix(set_y, results)
    sim = 0
    for i in range(len(results)):
        if set_y[i] == results[i]:
            sim += 1
    return round(sim/len(results),2)

train_accuracy = cal_acc(train_x,train_y)
test_accuracy = cal_acc(test_x,test_y)

print("train accurcy:", train_accuracy)
print("test accurcy:", test_accuracy)

train accurcy: 0.88
test accurcy: 0.45


In [35]:
text = """ak_parti Genel  Başkanı ve Cumhurbaşkanı Recep Tayyip Erdoğan, Hamas Siyasi Büro Başkanı İsmail Heniyye ile telefon görüşmesi gerçekleştirdi.
Erdoğan, Hamas lideri Heniyye iler yaptığı telefon görüşmesinde Heniyye'ye, ağabeyi Halid Heniyye’nin vefatı dolayısıyla başsağlığı diledi"""
text_vector = create_vector(text)

In [36]:
results = forest.predict([text_vector])
print(results)

['dunya ']


In [50]:
text_to_summur = """çekirge , pire ve uçan kaz bir gün saraya davet edilmişler . kral üçünün arasında bir yarış düzenleyecek ve en yükseğe sıçrayana büyük bir ödül verecekmiş . sonunda ödülü açıklamış . yarısı kazanana kızımı vereceğim demiş . yarışmaya önce pire , çekirge sonra da uçan kaz tek tek zıplayarak yarışmışlar . bunların her biri kendini diğerlerinden üstün görüyormuş . ilk yarışan pire çok yüksek zıplayınca görünmemiş ve onu almamış olarak kabul etmişler . çekirgede pirenin yarısı kadar zıplamış ancak kralın üstüne konduğu için kral ona çok kızmış . sıra uçan kaza gelmiş , kaz nazikçe prensesin yanına kadar sıçramış kral bu nazikçe sıçrayışı görünce kararını açıklamış . ' en yükseğe sıçrayan kızıma doğru sıçrayandır . ' demiştir ve prensesi uçan kaza vermeye karar vermiş . olayı duyan pire ile çekirge yaptıkları hatayı anlayıp çok üzülmüşler ."""


In [45]:
import nltk
from nltk import sent_tokenize


In [51]:

sents = sent_tokenize(text_to_summur)
print("we have", len(sents),"in the text")
print(sents)

we have 12 in the text
['çekirge , pire ve uçan kaz bir gün saraya davet edilmişler .', 'kral üçünün arasında bir yarış düzenleyecek ve en yükseğe sıçrayana büyük bir ödül verecekmiş .', 'sonunda ödülü açıklamış .', 'yarısı kazanana kızımı vereceğim demiş .', 'yarışmaya önce pire , çekirge sonra da uçan kaz tek tek zıplayarak yarışmışlar .', 'bunların her biri kendini diğerlerinden üstün görüyormuş .', 'ilk yarışan pire çok yüksek zıplayınca görünmemiş ve onu almamış olarak kabul etmişler .', 'çekirgede pirenin yarısı kadar zıplamış ancak kralın üstüne konduğu için kral ona çok kızmış .', "sıra uçan kaza gelmiş , kaz nazikçe prensesin yanına kadar sıçramış kral bu nazikçe sıçrayışı görünce kararını açıklamış . '", "en yükseğe sıçrayan kızıma doğru sıçrayandır . '", 'demiştir ve prensesi uçan kaza vermeye karar vermiş .', 'olayı duyan pire ile çekirge yaptıkları hatayı anlayıp çok üzülmüşler .']


In [52]:
sents_vecs = []
for sent in sents:
    sents_vecs.append(create_vector(sent))
print(sents_vecs)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [53]:
sums = []
for vec in sents_vecs:
    sums.append(sum(vec))

list_to_print = ["sent{}".format(i) for i in range(len(sents))]
print(list_to_print)
print(sums)

['sent0', 'sent1', 'sent2', 'sent3', 'sent4', 'sent5', 'sent6', 'sent7', 'sent8', 'sent9', 'sent10', 'sent11']
[0, 1, 0, 0, 0, 0, 2, 2, 0, 0, 0, 1]


In [54]:
idx = np.nonzero(sums)[0]
sents = np.array(sents)

print("summarization:")
print("-"*50)
print("\n".join(sents[idx]))

summarization:
--------------------------------------------------
kral üçünün arasında bir yarış düzenleyecek ve en yükseğe sıçrayana büyük bir ödül verecekmiş .
ilk yarışan pire çok yüksek zıplayınca görünmemiş ve onu almamış olarak kabul etmişler .
çekirgede pirenin yarısı kadar zıplamış ancak kralın üstüne konduğu için kral ona çok kızmış .
olayı duyan pire ile çekirge yaptıkları hatayı anlayıp çok üzülmüşler .
